# Voiceshell Work Notebook

*Adapted from Allison Parrish's Understanding Word Vectors notebook*

#### Python Package Installation for Speech Recognition

*Run in terminal to install SpeechRecognition.py & PocketSphinx engine:*

`pip install SpeechRecognition`

`pip install pocketsphinx`

#### Importing required libraries:

In [15]:
# IMPORTS
import math
import random
from __future__ import unicode_literals
import spacy
import speech_recognition as sr
import os
import numpy as np
from numpy import dot
from numpy.linalg import norm

#### Defining basic mathematical functions for vector math:

In [2]:
def meanv(coords):
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean

In [3]:
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

#### Defining Spacy/Word Vector low level functions:

In [4]:
def sentvec(s):
    sent = nlp(s)
    return meanv([w.vector for w in sent])

In [5]:
def spacy_closest_sent(space, input_str, n=10):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine(np.mean([w.vector for w in x], axis=0), input_vec),
                  reverse=True)[:n]

#### Creating Spacy processor and loading a corpus:

In [6]:
nlp = spacy.load('en')
doc = nlp(open("revisedCorpus/allPoems.txt").read())

#### Splitting corpus into individual, complete sentences:

In [7]:
sentences = list(doc.sents)

#### Define list of prompts to user:

In [8]:
inquiries = ["How are you feeling?", "Tell me a quote you like.", "What are the first three words that pop into your head?", "What day is it today?", "What's your favourite colour?", "Where are you right now?", "What's your job?", "What do you like to eat?", "What animal would you want as a pet?"]

#### Defining user input prompts, and getting input to process, return result:

In [9]:
print(inquiry)
user_input = input()

for sent in spacy_closest_sent(sentences, user_input):
    print("---")
    print(sent.text.replace('\n',' '))
    print("---")

NameError: name 'inquiry' is not defined

#### Setting up basic SpeechRecognition input:
[Source](http://www.codesofinterest.com/2017/03/python-speech-recognition-pocketsphinx.html)

In [22]:
# get microphone input
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Just a moment, calibrating the microphone...")
    # 5 second listen and find ambient noise level
    r.adjust_for_ambient_noise(source, duration=5)
    inquiry = random.choice(inquiries)
    
    # read question aloud and print it
    os.system("say '" + inquiry + "'")
    print(inquiry," (Speak aloud, please.)")
    audio = r.listen(source)
    
# recognise speech with Sphinx
try:
    recogSpeech = r.recognize_sphinx(audio) + "'"
    print("I heard you say: ", recogSpeech)
    for sent in spacy_closest_sent(sentences, recogSpeech):
        print("----------")
        print("...")
        print(sent.text)
        print("----------")
except sr.UnknownValueError:
    print("Sorry, I couldn't understand you.")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

Just a moment, calibrating the microphone...
What's your favourite colour?  (Speak aloud, please.)
I heard you say:  he's not the god'
----------
...
I like the fact

that he also has his mind on other things. 

----------
----------
...
‘That which is done

is that which shall be done’ 
is all very well in the- 
ory, but what if the sun 
were black, and the book dead 
wrong, and the interval under death 
demanded a father 
as unlike his father as day 
and night?
----------
----------
...
By this time I knew that he was nearly 
a man, and that if I didn’t cut his finger off 
or shoot him in the foot, he would go. 

----------
----------
...
C K Stead

 
  



Without

Crossing Cook Strait
going home to be
ordained in the

parish of his 
father, while seas wished
by and the wind

had its say in the
wires, it came to
him there was no

God.
----------
----------
...
Not that
God was sulking or had
turned His back—that

had happened 
often.
----------
----------
...
Funny how they grow up 

In [20]:
os.system("say '" + inquiry + "'")

0

#### Playing with Python WAV Audio playback

In [ ]:
import wave, sys, pyaudio


# sound = wave.open(audioFile)
sound = wave.open('PoemsAudio/Avia/avia0.wav')
p = pyaudio.PyAudio()
chunk = 1024
stream = p.open(format = p.get_format_from_width(sound.getsampwidth()), channels = sound.getnchannels(), rate = sound.getframerate(), output = True)
data = sound.readframes(chunk)
while data != '':
    stream.write(data)
    data = sound.readframes(chunk)